In [2]:
import pickle
from tqdm import tqdm
import pandas as pd
import json

%load_ext autoreload
%autoreload 2

In [3]:
import os 

os.chdir('c:/Users/Domen/IronHack/01_projects/IH_final_project_data-jobs')
print(os.getcwd())

from src.myModules import scraper_ds as mybib
from src.myModules import my_NLP as myNLP

%load_ext autoreload
%autoreload 2

c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Transform and Cluster Title using NLP

load cleaned dataframe

In [4]:
print(os.getcwd())
with open("config/config.json", 'r') as f:
    config = json.load(f)
    
dataframe_path = config["cleaned_data_path"]
NLP_data_path = config['NLP_data_path']


print(dataframe_path)
print(NLP_data_path)


cleaned_df_input =  'transformed_dataframe'
nlp_df_output = 'NLP_title'


c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
data/cleaned_data/
data/NLP_data/


In [5]:
with open(f"{dataframe_path}{cleaned_df_input}.pkl", "rb") as file:
    df = pickle.load(file)
    

### first cleaning

In [6]:
nlp_df = pd.DataFrame(df) # create a new data frame

In [7]:
nlp_df.head(10)

,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url,keyword,language
0,3685669741,Software Engineer Fullstack (m/w/d),Atruvia AG,"Aschheim, Bavaria, Germany",4 days ago,VollzeitWir sind der Digitalisierungspartner d...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-09-26,https://www.linkedin.com/jobs/search?keywords=...,data_engineer,de
1,3719620327,Data Analyst - Business Intelligence,Almedia,"Berlin, Berlin, Germany",1 week ago,Almedia helps companies grow by promoting thei...,Mid-Senior level,Full-time,Analyst,"Technology, Information and Internet",2023-09-26,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
2,3720365801,(Junior) Data Engineer (m/f/d),Sandbox Interactive,"Berlin, Berlin, Germany",4 days ago,YOUR GAMEBuild and maintain the current data i...,Mid-Senior level,Full-time,Information Technology,Computer Games,2023-09-26,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
3,3672508925,Financial Data Analyst (m/f/d),Ultramarin,"Berlin, Berlin, Germany",2 months ago,About The PositionWe are looking for a top-tie...,Mid-Senior level,Full-time,Information Technology,Financial Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
4,3695466781,Business Intelligence Analyst (f/m/d),Les Lunes,"Berlin, Berlin, Germany",4 weeks ago,Your missionAs a Business Intelligence Analyst...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",Retail Apparel and Fashion,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
5,3585738180,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,Orange Quarter are working with an exciting st...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
6,3585734564,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,Orange Quarter are currently looking for a Dat...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
7,3704916810,BI Analyst (f/m/d),Enter,"Berlin, Berlin, Germany",3 weeks ago,Build the future with Enter.Our mission is to ...,Mid-Senior level,Full-time,Information Technology,Architecture and Planning,2023-09-26,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
8,3585734595,Data Analyst,Orange Quarter,"Berlin, Berlin, Germany",5 months ago,We are currently working with an emerging SaaS...,Entry level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en
9,3698396795,Senior Data Analyst,Klim,"Berlin, Berlin, Germany",1 month ago,"We are Klim, a Berlin-based clean-tech company...",Mid-Senior level,Full-time,Other,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst,en


In [8]:
tqdm.pandas(desc="Processing")
nlp_df['cp1_title'] = nlp_df["title"].progress_apply(myNLP.clean_up)
nlp_df['cp2_title'] = nlp_df['cp1_title'].progress_apply(myNLP.tokenize)

Processing: 100%|██████████| 29302/29302 [00:02<00:00, 14397.21it/s]


In [9]:
nlp_df = nlp_df.progress_apply(myNLP.extract_level, axis=1)
nlp_df['first_match'] = None



Processing: 100%|██████████| 29302/29302 [00:15<00:00, 1939.63it/s]


In [10]:
nlp_df['first_match'].value_counts(dropna=False)

first_match
None    29302
Name: count, dtype: int64

In [11]:
nlp_df = nlp_df.progress_apply(myNLP.get_job_title, axis=1)
nlp_df[nlp_df['new_job_title'].isna()] = nlp_df[nlp_df['new_job_title'].isna()].progress_apply(myNLP.second_round_jobtitle, axis=1)
nlp_df[nlp_df['new_job_title'].isna()] = nlp_df[nlp_df['new_job_title'].isna()].progress_apply(myNLP.false_col, axis=1)
nlp_df[nlp_df['job_level'].isin(['senior_level', 'student_level', 'manager_level'])] = nlp_df[nlp_df['job_level'].isin(['senior_level', 'student_level', 'manager_level'])].progress_apply(myNLP.false_col, axis=1)



Processing: 100%|██████████| 11388/11388 [00:02<00:00, 5108.25it/s]


In [12]:
nlp_df['first_match'].value_counts(dropna=False)


first_match
False    19575
True      9727
Name: count, dtype: int64

In [13]:
with open(f"{NLP_data_path}{nlp_df_output}.pkl", "wb") as file:
    pickle.dump(nlp_df, file)

remove text in parentheses

In [30]:
pattern_to_extract = r'\((.*?)\)'

# Extract text within parentheses and put it in a new column
nlp_df['text_in_parentheses'] = nlp_df['cp1_title'].progress_apply(lambda x: re.findall(pattern_to_extract, x))

# Convert lists with only one element to that element, and keep lists for multiple elements
nlp_df['text_in_parentheses'] = nlp_df['text_in_parentheses'].progress_apply(lambda x: x[0] if len(x) == 1 else x)

# Remove the text within parentheses 
nlp_df['cp1_title'] = nlp_df['cp1_title'].progress_apply(lambda x: re.sub(pattern_to_extract, '', x).strip())


### Tokenize, lemmatize, remove stopwords

In [31]:
nlp_df['cp2_title'] = nlp_df['cp1_title'].progress_apply(myNLP.tokenize)
nlp_df['cp2_title'] = nlp_df['cp2_title'].progress_apply(myNLP.stem_and_lemmatize)
nlp_df['cp2_title'] = nlp_df['cp2_title'].progress_apply(myNLP.remove_stopwords)
nlp_df['cp2_title'] = nlp_df['cp2_title'].progress_apply(myNLP.re_blob) 


In [14]:
print(nlp_df.shape)
nlp_df_split = nlp_df[nlp_df['new_job_title'].isna()]

print(nlp_df_split.shape)

all_text = nlp_df_split['cp2_title']

bag = " ".join(all_text) # combining all job title 

(29302, 19)
(15339, 19)


In [15]:
nlp_df_split['new_job_title'].value_counts(dropna=False).head(50)

new_job_title
NaN    15339
Name: count, dtype: int64

In [34]:
# nlp_df_split = nlp_df_split.progress_apply(myNLP.second_round_jobtitle, axis=1)


Processing: 100%|██████████| 15197/15197 [00:03<00:00, 3910.32it/s]


In [35]:
# nlp_df_split['new_job_title'].value_counts(dropna=False).head(50)

new_job_title
NaN    15197
Name: count, dtype: int64

In [36]:
# nlp_df_split[['title','cp1_title', 'cp2_title', 'new_job_title']].head(50)

,title,cp1_title,cp2_title,new_job_title
43,(Junior) Consultant Intelligent Analytics and ...,(junior) consultant intelligent analytics and ...,() intelligent analytics and insights in berlin,NaN
44,Consultant - Digital Analytics (m/w/d) for Berlin,consultant digital analytics for berlin,digital analytics for berlin,NaN
48,Data Center Engineer L2 (On Demand),data center engineer l2 (on demand),data center engineer l2 (on demand),NaN
54,Associate - Research (w/m/x/k.A.),associate research (w m x k.a.),associate research (w m x k.a.),NaN
65,Junior User Researcher (m/f/d),junior user researcher,user researcher,NaN
71,"Associate Researcher, Electric Vehicles and Eq...",associate researcher electric vehicles and equ...,associate researcher electric vehicles and equ...,NaN
73,Market Research industry - Automotive/Sports c...,market research industry automotive sports ca...,market research industry automotive sports ca...,NaN
75,Sales Operations & Enablement (f/m/x),sales operations enablement,sales operations enablement,NaN
77,Strategy Manager,strategy manager,strategy,NaN
80,Commercial Strategy Manager,commercial strategy manager,commercial strategy,NaN


In [37]:
# filtered_df = nlp_df_split[nlp_df_split['cp1_title'].str.contains("software_engineer", case=False)]
# filtered_df[['title','cp1_title', 'cp2_title', 'new_job_title']]

,title,cp1_title,cp2_title,new_job_title


import re

In [38]:
# import re

In [39]:
# test_title = filtered_df.loc[0, 'cp1_title']
# test_title

In [40]:
# # keywords = ['data_analyst', 'software_architect', 'ml_engineer', 'it_systemadmin', 'software_developer','scientist_','research_scientist', 'data_architect','java_software_engineer','data_engineer', 'database_datawarehouse', 'business_analyst', 'product_analyst', 
# #                 'business_intelligence_analyst', 'full_stack', 'database_administrator', 'fp&a_analyst', 'product_analyst'
# #                 'programm_manager', 'data_analytics', 'big_data_engineer/specialist', 'sap_specialist', 'devops_engineer', 'backend_developer', 'data_management'
# #                ' software_engineer',
# # 'analysis_engineer', 'data_scientist', 'machine_learning_engineer', 'ai_engineer', 'controlling_', 'it_systemadmin'
# #                'cloud_engineer', 'deep_learning', 'reporting_analyst', 'test_engineer', 'system_engineer', 'specialist', 'sap_specialist']


# keywords = ['data_analyst', 'software_architect', 'ml_engineer', 'it_systemadmin', 'software_developer','scientist_','research_scientist', 'data_architect','java_software_engineer','data_engineer', 'database_datawarehouse', 'business_analyst', 'product_analyst', 
#             'business_intelligence_analyst', 'full_stack', 'database_administrator', 'fp&a_analyst', 'product_analyst',
#             'programm_manager', 'data_analytics', 'big_data_engineer/specialist', 'sap_specialist', 'devops_engineer', 'backend_developer', 'data_management',
#             'software_engineer', 'analysis_engineer', 'data_scientist', 'machine_learning_engineer', 'ai_engineer', 'controlling_', 'it_systemadmin',
#             'cloud_engineer', 'deep_learning', 'reporting_analyst', 'test_engineer', 'system_engineer', 'specialist', 'sap_specialist']


# regex_pattern = '|'.join(re.escape(keyword) for keyword in keywords)

# matched_keywords = re.findall(regex_pattern, test_title, re.IGNORECASE)

# matched_keywords

In [41]:
# test_df = test_df.progress_apply(myNLP.get_job_title, axis=1)
# test_df

In [42]:
# myNLP.word_cloud(nlp_df_split, 'cp1_title')

In [44]:
# from sklearn.metrics import silhouette_score
# import matplotlib.pyplot as plt

# # Initialize variables
# silhouette_scores = []
# inertia = []
# # Loop through different numbers of clusters
# for i in range(10, 30):  # Start from 2 as silhouette_score needs at least 2 clusters
#     kmeans = KMeans(n_clusters=i, init='k-means++')
#     kmeans.fit(X)
#     cluster_labels = kmeans.labels_
    
#     inertia.append(kmeans.inertia_)
    
#     # Calculate the silhouette score and append to the list
#     silhouette_avg = silhouette_score(X, cluster_labels)
#     silhouette_scores.append(silhouette_avg)
#     # print(f"For n_clusters = {i}, the silhouette score is {silhouette_avg}")

In [45]:

# # Plotting the silhouette scores
# plt.plot(range(10, 30), silhouette_scores)
# plt.title('Silhouette Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('Silhouette Score')
# plt.show()



# plt.plot(range(10, 30), inertia)
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('Inertia')
# plt.show()

In [46]:
# # Step 3: Clustering
# kmeans = KMeans(n_clusters=28)  # Assuming 5 clusters
# kmeans.fit(X)
# nlp_df_split['cluster_kmeans'] = kmeans.labels_

In [47]:
# kmean_groups = nlp_df_split[['id', 'cp1_title', 'cp2_title', 'cp3_title', 'cluster_kmeans']]

# with open("webscraper/model_data/kmean_groups_v3.csv", "wb") as file:
#     kmean_groups.to_csv(file)

In [1]:
nlp_df['new_job_title']

NameError: name 'nlp_df' is not defined